In [1]:
import pandas as pd
import requests
from splinter import Browser
from bs4 import BeautifulSoup
import time

In [2]:
# Yelp url for restaurants in Morris County, NJ to be scraped
url = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Morris+County%2C+NJ&ns=1'
response = requests.get(url)

In [3]:
# intialize variables and create empty lists
nextpage_link = True
restaurant_name = []
restaurant_url = []
restaurant_rating = []
price_cuisine = []
 
while (nextpage_link):
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # class for restaurant details
    containers = soup.find_all('div', class_='scrollablePhotos__09f24__1PpB8')[1:]
    
    # scroll through each container
    for item in containers:
        #print(item)
        # find and add restaurant name to list
        rest_name = item.find('a',class_='link__09f24__1kwXV')
        restaurant_name.append(rest_name.text)
        
        # find and add restaurant url to list
        restaurant_url.append("https://www.yelp.com" + rest_name['href'])
        
        # find and add rating (if exists) to list
        rating = item.find('div',class_='i-stars__09f24__1T6rz')
        if rating:
            restaurant_rating.append(rating['aria-label'].split(' ')[0])
        else:
            restaurant_rating.append("")
        
        # find and add cuisine to list
        price_cuisine.append(item.find('div',class_='priceCategory__09f24__2IbAM').text)

    # find the next page url
    next_page = soup.find('a', class_='next-link')
    
    # if there is a another page, assign the url, otherwise exit
    if next_page:        
        nextpage_url = next_page['href']
        response = requests.get(nextpage_url)
        #time.sleep(2)
    else:
        nextpage_link = False

# Create Dataframe
yelp_df = pd.DataFrame({'restaurant_name': restaurant_name,
                       'restaurant_url': restaurant_url,
                       'restaurant_rating': restaurant_rating,
                       'price_cuisine': price_cuisine})

yelp_df    

,restaurant_name,restaurant_url,restaurant_rating,price_cuisine
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces"
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4,"$$American (New), Bars, Pizza"
2,Missy’s Main Street Cafe,https://www.yelp.com/biz/missys-main-street-ca...,5,"$Breakfast & Brunch, Cafes"
3,The Corner Bistro,https://www.yelp.com/biz/the-corner-bistro-ran...,4,"$$Diners, Italian"
4,Beanbury,https://www.yelp.com/biz/beanbury-succasunna?o...,5,"Coffee & Tea, Breakfast & Brunch"
...,...,...,...,...
235,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps"
236,LongHorn Steakhouse,https://www.yelp.com/biz/longhorn-steakhouse-f...,3.5,"$$Steakhouses, American (Traditional), Barbeque"
237,Hunan Chinese Room,https://www.yelp.com/biz/hunan-chinese-room-mo...,3.5,$$Chinese
238,Brenda Lee Restaurant,https://www.yelp.com/biz/brenda-lee-restaurant...,3.5,Mexican


In [4]:
# create empty lists to hold address
address1 = []
address2 = []

executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser =  Browser("chrome", **executable_path, headless=False)

# iterate through all the rows in the yelp dataframe
for index, row in yelp_df.iterrows():
    
    # visit each restaurant's url
    browser.visit(row['restaurant_url'])

    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    # find the address tag
    address = soup.find('address').find_all('span', class_='raw__373c0__3rcx7')
    
    # find address1 and add to list
    street_address = address[0].text
    for i in range(1, len(address)-1):
        street_address = street_address + address[i].text + " "
    address1.append(street_address)
    
    # find the rest of the address
    address2 = address[len(address)-1].text
    
browser.quit()

# add address columns to yelp dataframe
yelp_df['address1'] = address1
yelp_df['address2'] = address2
                  
yelp_df

,restaurant_name,restaurant_url,restaurant_rating,price_cuisine,address1,address2
0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869"
1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869"
2,Missy’s Main Street Cafe,https://www.yelp.com/biz/missys-main-street-ca...,5,"$Breakfast & Brunch, Cafes",181 E Main St,"Randolph, NJ 07869"
3,The Corner Bistro,https://www.yelp.com/biz/the-corner-bistro-ran...,4,"$$Diners, Italian",477 NJ-10,"Randolph, NJ 07869"
4,Beanbury,https://www.yelp.com/biz/beanbury-succasunna?o...,5,"Coffee & Tea, Breakfast & Brunch",37 Route 10 E,"Randolph, NJ 07869"
...,...,...,...,...,...,...
235,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Randolph, NJ 07869"
236,LongHorn Steakhouse,https://www.yelp.com/biz/longhorn-steakhouse-f...,3.5,"$$Steakhouses, American (Traditional), Barbeque",50-J International Dr S,"Randolph, NJ 07869"
237,Hunan Chinese Room,https://www.yelp.com/biz/hunan-chinese-room-mo...,3.5,$$Chinese,255 Speedwell Ave,"Randolph, NJ 07869"
238,Brenda Lee Restaurant,https://www.yelp.com/biz/brenda-lee-restaurant...,3.5,Mexican,15 E Blackwell St,"Randolph, NJ 07869"


In [5]:
# save to csv file
yelp_df.to_csv('../Resources/yelp_data.csv')

In [ ]:
json_file = "../Resources/yelp_academic_dataset_business.json"
yelp_original_df = pd.read_json(json_file, lines=True)
yelp_original_df.head()

yelp_original_df.groupby('state')['state'].count().sort_values(ascending=False)

In [ ]:
yelp_original_df.groupby('city')['city'].count().sort_values(ascending=False)